In [1]:
import cv2
import mediapipe as mp
import time

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Define the y-coordinate thresholds for the "down" and "up" wrist positions
DOWN_POS_THRESHOLD = 0.4
UP_POS_THRESHOLD = 0.7

# Initialize pull-up counting variables
pull_up_count = 0
is_pull_up = False

cap = cv2.VideoCapture(0)
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            continue

        # Convert the BGR image to RGB before processing
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Process the image with the pose model
        results = pose.process(image)

        # Draw the pose landmarks on the image
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            # Get the position of the right wrist and shoulder landmarks
            right_wrist_pos = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST].y
            right_shoulder_pos = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y
            left_wrist_pos= results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST].y
            left_shoulder_pos = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y

            # Check if the right wrist is in the "down" position for a push-up
            if right_wrist_pos < right_shoulder_pos * DOWN_POS_THRESHOLD and left_wrist_pos < left_shoulder_pos * DOWN_POS_THRESHOLD:
                is_pull_up = True

            # If the wrist was in the "down" position and is now in the "up" position,
            # increment the push-up count
            if is_pull_up and right_wrist_pos > right_shoulder_pos * UP_POS_THRESHOLD and left_wrist_pos > left_shoulder_pos * UP_POS_THRESHOLD:
                pull_up_count += 1
                is_pull_up = False

        # Draw the push-up count on the image
        font = cv2.FONT_HERSHEY_SIMPLEX
        org = (50, 100)
        font_scale = 2
        color = (255, 0, 0)
        thickness = 3
        image = cv2.putText(image, "Pull-up count: " + str(pull_up_count), org, font, font_scale, color, thickness, cv2.LINE_AA)

        # Show the image and wait for a key press
        cv2.imshow('MediaPipe Pose', image)
        if cv2.waitKey(1) & 0xFF == 27:
            break

    cap.release()
    cv2.destroyAllWindows()